# Plot Math Primitives

> Mathematical primitives for HexMagic plotting

In [ ]:
#| default_exp plot.primitives

In [ ]:
#| export
import numpy as np
import math
import random
from dataclasses import dataclass
from fastcore.basics import patch
from bezier_interpolation import cubic_interpolation
from HexMagic.styles import StyleCSS, indent

In [ ]:
#| export
class PrimitiveDemo:
    def __init__(self):
        self.help = ""

In [ ]:
#| export

@dataclass(frozen=True)
class MapCord:
    x: float
    y: float

    def __str__(self):
        return f"({self.x},{self.y})"

    def __repr__(self):
        return f"({self.x},{self.y})"

    def midPoint(self,other):
        return MapCord((self.x + other.x)/2,(self.y + other.y)/2)

    def angle(self,other):
        return math.atan2(other.y - self.y, other.x - self.x)

    def toPoint(self,other):
        return MapCord(other.x - self.x ,other.y - self.y)

    def to_csv(self):
        return f"{self.x}^{self.y}"

    def from_csv( s:str):
        vals = s.split('^')
        return MapCord(x=float(vals[0]), y=float(vals[1]))

    def distance(self,other):
        vector =  self.toPoint(other)
        return math.sqrt(vector.x * vector.x  + vector.y * vector.y)

    def midOff(self,other,distance):
        """Calculate a point perpendicular to the line segment between two points.
    
    This function finds the midpoint between self and other, then moves 
    perpendicular to the line by the specified distance. Useful for creating
    offset curves and adding variation to straight lines (e.g., river meandering).
    
    Args:
        other (MapCord): The other endpoint of the line segment
        distance (float): How far to offset perpendicular to the line (can be negative)
        
    Returns:
        MapCord: A point perpendicular to the midpoint at the given distance,
                or MapCord(-1, -1) if self and other are identical
    
    Example:
        p1 = MapCord(0, 0)
        p2 = MapCord(10, 0)
        offset = p1.midOff(p2, 5)  # Returns point above/below midpoint
    """
        length = self.distance(other)
        if length == 0:
            return MapCord(-1,-1)

        delta = self.toPoint(other)
        mid = self.midPoint(other)
        
            # Perpendicular unit vector
        Ux = -delta.y / length
        Uy = delta.x / length
        return MapCord (mid.x + Ux * distance, mid.y + Uy * distance)

In [ ]:
#| export
@patch
def __lt__(self: MapCord, other: MapCord) -> bool:
    """Less than comparison: first by x, then by y."""
    if self.x != other.x:
        return self.x < other.x
    return self.y < other.y

In [ ]:
#| export
@dataclass(frozen=True)
class MapSize:
    width: float
    height: float
    def background(self,style):
        return f"<rect width=\"{self.width}\" height=\"{self.height }\" class =\"" + style.name+ "\"  />"

    def to_csv(self):
        return f"{self.width}^{self.height}"
 
    def from_csv( s:str):
        vals = s.split('^')
        return MapSize(width=float(vals[0]), height=float(vals[1]))

In [ ]:
#| export
@dataclass(frozen=True)
class MapRect:
    origin: MapCord
    dimensons: MapSize


    def minX(self):
        return self.origin.x

    def minY(self):
        return self.origin.y

    def maxX(self):
        return self.origin.x + self.dimensons.width

    def maxY(self):
        return self.origin.y + self.dimensons.height

    def to_csv(self):
        return f"{self.origin.to_csv()}&{self.dimensons.to_csv()}"
 
    def from_csv( s:str):
        vals = s.split('&')
        return  MapRect(origin=MapCord.from_csv(vals[0]),dimensons=MapSize.from_csv(vals[1]))


In [ ]:
#| export
def MakeCord(x):
    return MapCord(x[0],x[1])
    
def MakeSize(x):
    return MapSize(x[0],x[1])

In [ ]:
aChord = MapCord(4,5)
bChord = MapCord.from_csv(aChord.to_csv())
print(bChord.to_csv(),aChord.to_csv(),bChord,aChord)
aChord

4.0^5.0 4^5 (4.0,5.0) (4,5)


(4,5)

## Path

In [ ]:
#| export

class MapPath:
    def __init__(self,points,style = StyleCSS("blank")):

        self.points = [x if isinstance(x, MapCord) else MakeCord(x) for x in points]
        self.style = style
    

    def drawPolygon(self , adds = ""):
        path_data = "\n<path d=\""
        for i, point in enumerate(self.points):
            if i == 0:
                path_data += f"M {point.x:.1f} {point.y:.1f}"
            else:
                path_data += (f" L {point.x:.1f} {point.y:.1f}")
        path_data += "\" class =\"" + self.style.name + f"\"  {adds} />\n"
        return path_data
    
    def drawSpline(self,adds = ""):
        path_spline = "\n<path d=\""
        xN = np.array([])
        yN = np.array([])
        
        for i, point in enumerate(self.points):
            xN = np.append(xN, point.x)
            yN = np.append(yN, point.y)
    
            if i == 0:
                path_spline += f"M {point.x:.1f} {point.y:.1f}"
    
        c_data = {"x": xN, "y": yN}
        c_data = list(zip(c_data["x"], c_data["y"]))
        c_interpolated_data = cubic_interpolation(c_data)

        line = ""

        for i, point in enumerate(c_interpolated_data):
            x, y = point
            if i > 0:
                if i % 3 == 1:
                    line += f" C"
                line += f" {x:.1f} {y:.1f}"
            if len(line)>50:
                path_spline += line + "\n\t"
                line = ""

        path_spline += line
        
        if len(adds) > 2:
            path_spline += f"\"  {adds} />\n"

        else:
            path_spline += "\" class =\"" + self.style.name+ f"\" />\n"
        return indent(path_spline)

    def svg(self,adds = ""):
        return self.drawSpline(adds)

    def from_boundary( points: list[tuple[float, float]], style=None) -> 'MapPath':
        """Factory method to create a closed boundary path from coordinate tuples.
        
        Args:
            points: List of (x, y) tuples
            style: Optional StyleCSS, defaults to blank
            
        Returns:
            MapPath with points converted to MapCord
        """
        if style is None:
            style = StyleCSS("blank")
        path = MapPath(points, style)
        return path.closed()  # Use existing closed() method

    # Also add to_svg_path for easy SVG generation 
    def to_svg_path(self, close: bool = True) -> str:
        """Generate just the path 'd' attribute string."""
        if not self.points:
            return ""
        
        path_data = f"M {self.points[0].x:.1f},{self.points[0].y:.1f}"
        for point in self.points[1:]:
            path_data += f" L {point.x:.1f},{point.y:.1f}"
        
        if close:
            path_data += " Z"
        
        return path_data

In [ ]:
#| export
@patch
def length(self: MapPath) -> float:
    """Calculate total path length"""
    total = 0
    for i in range(len(self.points) - 1):
        total += self.points[i].distance(self.points[i + 1])
    return total

@patch
def reverse(self: MapPath) -> MapPath:
    """Return reversed path"""
    return MapPath(list(reversed(self.points)), self.style)

In [ ]:
#| export
@patch
def subsample(self: MapPath, num_points: int) -> MapPath:
    """Resample path to have specific number of points"""
    if num_points < 2 or len(self.points) < 2:
        return self
    
    total_length = self.length()
    segment_length = total_length / (num_points - 1)
    
    new_points = [self.points[0]]
    current_dist = 0
    target_dist = segment_length
    
    for i in range(len(self.points) - 1):
        p1 = self.points[i]
        p2 = self.points[i + 1]
        seg_length = p1.distance(p2)
        
        while target_dist <= current_dist + seg_length:
            t = (target_dist - current_dist) / seg_length
            new_points.append(p1.lerp(p2, t))
            target_dist += segment_length
            
        current_dist += seg_length
    
    new_points.append(self.points[-1])
    return MapPath(new_points[:num_points], self.style)

In [ ]:
#| export
@patch
def closed(self: MapPath) -> MapPath:
    """Return closed version of path (adds first point to end if not already closed)"""
    if self.points[0].distance(self.points[-1]) < 0.01:
        return self
    return MapPath(self.points + [self.points[0]], self.style)

In [ ]:
#| export
@patch
def smooth(self: MapPath, iterations: int = 1) -> MapPath:
    """Smooth path using average of neighbors"""
    if len(self.points) < 3:
        return self
    
    points = list(self.points)
    for _ in range(iterations):
        new_points = [points[0]]
        for i in range(1, len(points) - 1):
            avg_x = (points[i-1].x + points[i].x + points[i+1].x) / 3
            avg_y = (points[i-1].y + points[i].y + points[i+1].y) / 3
            new_points.append(MapCord(avg_x, avg_y))
        new_points.append(points[-1])
        points = new_points
    
    return MapPath(points, self.style)

In [ ]:
#| export
@patch
def add_noise(self: MapPath, amount: float, seed: int = None) -> MapPath:
    """Add random noise to path for organic appearance"""
    if seed is not None:
        random.seed(seed)
    
    new_points = []
    for point in self.points:
        offset_x = (random.random() - 0.5) * amount * 2
        offset_y = (random.random() - 0.5) * amount * 2
        new_points.append(MapCord(point.x + offset_x, point.y + offset_y))
    
    return MapPath(new_points, self.style)

In [ ]:
#| export
@patch
def make_windy(self: MapPath, iterations: int = 1, offset_factor: float = 0.15, 
               seed: int = None, vary_direction: bool = True) -> MapPath:
    """Make path more windy by recursively adding perpendicular offsets at midpoints.
    
    This method works by:
    1. For each pair of consecutive points, find the midpoint
    2. Calculate a perpendicular offset at that midpoint
    3. Insert the offset point into the path
    4. Repeat for specified number of iterations
    
    Args:
        iterations: Number of subdivision iterations (more = more curves)
        offset_factor: How far to offset as fraction of segment length (0.0-0.5)
                      Default 0.15 gives gentle curves, 0.3+ gives dramatic curves
        seed: Random seed for reproducible results. If None, uses alternating offsets
        vary_direction: If True and seed is None, alternates offset direction for
                       natural meandering. If False, offsets are consistent.
    
    Returns:
        New MapPath with windy characteristics
    
    Example:
        # Gentle meandering river
        river = MapPath([(0, 0), (100, 0), (200, 50)], river_style)
        windy_river = river.make_windy(iterations=2, offset_factor=0.2)
        
        # Dramatic serpentine path
        snake = path.make_windy(iterations=3, offset_factor=0.35, seed=42)
    """
    if len(self.points) < 2:
        return self
    
    if seed is not None:
        random.seed(seed)
    
    points = list(self.points)
    
    for iteration in range(iterations):
        new_points = [points[0]]  # Keep first point
        
        for i in range(len(points) - 1):
            p1 = points[i]
            p2 = points[i + 1]
            
            # Calculate offset distance based on segment length
            segment_length = p1.distance(p2)
            offset_distance = segment_length * offset_factor
            
            # Determine offset direction
            if seed is not None:
                # Random offset direction
                offset_distance *= random.choice([-1, 1])
            elif vary_direction:
                # Alternate direction for natural meandering
                # Use both iteration and segment index for variation
                direction = 1 if ((i + iteration) % 2 == 0) else -1
                offset_distance *= direction
            # else: consistent direction (always positive)
            
            # Use the existing midOff method which calculates perpendicular offset
            mid_point = p1.midOff(p2, offset_distance)
            
            # Handle edge case where points are identical
            if mid_point.x == -1 and mid_point.y == -1:
                mid_point = p1.midPoint(p2)
            
            new_points.append(mid_point)
            new_points.append(p2)
        
        points = new_points
    
    return MapPath(points, self.style)

In [ ]:
#| export
@patch
def make_windy_variable(self: MapPath, iterations: int = 1, 
                         offset_min: float = 0.05, offset_max: float = 0.25,
                         seed: int = None) -> MapPath:
    """Make path windy with variable offset amounts for more organic appearance.
    
    Similar to make_windy but each subdivision uses a random offset factor
    within the specified range, creating more natural variation.
    
    Args:
        iterations: Number of subdivision iterations
        offset_min: Minimum offset as fraction of segment length
        offset_max: Maximum offset as fraction of segment length
        seed: Random seed for reproducible results
    
    Returns:
        New MapPath with organic windy characteristics
    """
    if len(self.points) < 2:
        return self
    
    if seed is not None:
        random.seed(seed)
    
    points = list(self.points)
    
    for iteration in range(iterations):
        new_points = [points[0]]
        
        for i in range(len(points) - 1):
            p1 = points[i]
            p2 = points[i + 1]
            
            segment_length = p1.distance(p2)
            
            # Random offset factor for this segment
            offset_factor = random.uniform(offset_min, offset_max)
            offset_distance = segment_length * offset_factor * random.choice([-1, 1])
            
            mid_point = p1.midOff(p2, offset_distance)
            
            if mid_point.x == -1 and mid_point.y == -1:
                mid_point = p1.midPoint(p2)
            
            new_points.append(mid_point)
            new_points.append(p2)
        
        points = new_points
    
    return MapPath(points, self.style)

In [ ]:
#| export
@patch
def drawClosed(self: MapPath, adds: str = "") -> str:
    """Draw closed polygon"""
    path_data = "\n<path d=\""
    for i, point in enumerate(self.points):
        if i == 0:
            path_data += f"M {point.x:.1f} {point.y:.1f}"
        else:
            path_data += f" L {point.x:.1f} {point.y:.1f}"
    path_data += " Z"  # Close path
    path_data += f"\" class=\"{self.style.name}\" {adds} />\n"
    return path_data

In [ ]:
#| export
@patch
def bounds(self: MapPath) -> MapRect:
    """Get bounding rectangle of path"""
    if not self.points:
        return MapRect(MapCord(0, 0), MapSize(0, 0))
    
    min_x = min(p.x for p in self.points)
    max_x = max(p.x for p in self.points)
    min_y = min(p.y for p in self.points)
    max_y = max(p.y for p in self.points)
    
    return MapRect(
        MapCord(min_x, min_y),
        MapSize(max_x - min_x, max_y - min_y)
    )

In [ ]:
#| export
@patch
def with_arrowhead(self: MapPath, arrow_size: float = 10, arrow_angle: float = 25, shouldFill = True) -> str:
    """Draw path with an arrowhead at the end.
    
    Args:
        arrow_size: Length of the arrowhead lines
        arrow_angle: Angle of arrowhead in degrees (from path direction)
    
    Returns:
        SVG path string including the arrowhead
    """
    if len(self.points) < 2:
        return self.svg()
    
    # Get the last two points to determine arrow direction
    p1 = self.points[-2]
    p2 = self.points[-1]
    
    # Calculate angle of the line
    angle = p1.angle(p2)
    
    # Convert arrow angle to radians
    arrow_rad = math.radians(arrow_angle)
    
    # Calculate the two arrowhead points
    left_angle = angle + math.pi - arrow_rad
    right_angle = angle + math.pi + arrow_rad
    
    left_point = MapCord(
        p2.x + arrow_size * math.cos(left_angle),
        p2.y + arrow_size * math.sin(left_angle)
    )
    
    right_point = MapCord(
        p2.x + arrow_size * math.cos(right_angle),
        p2.y + arrow_size * math.sin(right_angle)
    )
    
    # Draw main path
    result = self.svg()
    if shouldFill:
        stroke = self.style.properties["stroke"]
        result += f'\n<polygon points="{left_point.x:.1f},{left_point.y:.1f} {p2.x:.1f},{p2.y:.1f} {right_point.x:.1f},{right_point.y:.1f}" class="{self.style.name}"  style="fill: {stroke};"/>'

        

    else:
        
        # Add arrowhead lines
        result += f'\n<polygon points="{left_point.x:.1f},{left_point.y:.1f} {p2.x:.1f},{p2.y:.1f} {right_point.x:.1f},{right_point.y:.1f}" class="{self.style.name}" />'

    
    return result

In [ ]:
#| export
@patch
def __lt__(self: MapPath, other: MapPath) -> bool:
    """Less than comparison: first by x, then by y."""
    return len(self.points) < len(other.points)


In [ ]:
#| export
@patch
def demoLine(self:PrimitiveDemo):
 
    bMaker = SVGBuilder()
    bMaker.height = 70
    bMaker.width = 70
    lineStyle = StyleCSS("simpleLine",fill="#007fff",stroke= "#fff007",stroke_width=6)
    aStroke = MapPath([[0,4],[50,60],[0,40],[0,8]],lineStyle)
    bMaker.add_style(lineStyle)
    body = aStroke.svg()
    bMaker.updateLayers([body])
    
    return bMaker.show()


In [ ]:
from HexMagic.styles import SVGBuilder
PrimitiveDemo().demoLine()